# Import

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import category_encoders as ce

# Load

In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df = pd.concat([df_train, df_test]).reset_index(drop=True)

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


# EDA

In [3]:
pd.concat([
    pd.concat({
        "unique": df.nunique(),
        "missing": df.isnull().sum(),
        "dtype": df.dtypes
    }, axis=1),
    df.describe().T
], axis=1)

,unique,missing,dtype,count,mean,std,min,25%,50%,75%,max
PassengerId,1309,0,int64,1309.0,655.000000,378.020061,1.00,328.0000,655.0000,982.000,1309.0000
Survived,2,418,float64,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.000,1.0000
Pclass,3,0,int64,1309.0,2.294882,0.837836,1.00,2.0000,3.0000,3.000,3.0000
Name,1307,0,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,2,0,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,98,263,float64,1046.0,29.881138,14.413493,0.17,21.0000,28.0000,39.000,80.0000
SibSp,7,0,int64,1309.0,0.498854,1.041658,0.00,0.0000,0.0000,1.000,8.0000
Parch,8,0,int64,1309.0,0.385027,0.865560,0.00,0.0000,0.0000,0.000,9.0000
Ticket,929,0,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,281,1,float64,1308.0,33.295479,51.758668,0.00,7.8958,14.4542,31.275,512.3292


## Fare

欠損している人において、Pclass=3 & Embarked=S  
なので、この母集団における Fare の median で missing value を補完

In [4]:
df[df["Fare"].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [5]:
df['Fare'].fillna(df.query('Pclass==3 & Embarked=="S"')['Fare'].median(), inplace=True)

## Age

面倒なので mean で補完

In [6]:
df["Age"].fillna(df["Age"].mean(), inplace=True)

# Preprocess

## Ordinal Encoding

In [7]:
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


## One-Hot Encoding

In [9]:
ohe_columns = [
    "Pclass",
    "Embarked"
]

In [10]:
ohe = ce.OneHotEncoder(cols=ohe_columns, handle_unknown='impute')

In [11]:
df = ohe.fit_transform(df)

df.head()

,PassengerId,Survived,Pclass_1,Pclass_2,Pclass_3,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_1,Embarked_2,Embarked_3,Embarked_4
0,1,0.0,1,0,0,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,1,0,0,0
1,2,1.0,0,1,0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0,1,0,0
2,3,1.0,1,0,0,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,0,0
3,4,1.0,0,1,0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,1,0,0,0
4,5,0.0,1,0,0,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,1,0,0,0


## Scaling

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
sc_columns = [
    "Age",
    "Fare"
]

In [14]:
sc = StandardScaler()

df[sc_columns] = sc.fit_transform(df[sc_columns])
df.head()

,PassengerId,Survived,Pclass_1,Pclass_2,Pclass_3,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_1,Embarked_2,Embarked_3,Embarked_4
0,1,0.0,1,0,0,"Braund, Mr. Owen Harris",0,-0.611972,1,0,A/5 21171,-0.503176,NaN,1,0,0,0
1,2,1.0,0,1,0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,0.630431,1,0,PC 17599,0.734809,C85,0,1,0,0
2,3,1.0,1,0,0,"Heikkinen, Miss. Laina",1,-0.301371,0,0,STON/O2. 3101282,-0.490126,NaN,1,0,0,0
3,4,1.0,0,1,0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,0.397481,1,0,113803,0.383263,C123,1,0,0,0
4,5,0.0,1,0,0,"Allen, Mr. William Henry",0,0.397481,0,0,373450,-0.487709,NaN,1,0,0,0


## drop

In [15]:
df.drop([
    "Name",
    "Ticket",
    "Cabin"
], axis=1, inplace=True)

df.head()

,PassengerId,Survived,Pclass_1,Pclass_2,Pclass_3,Sex,Age,SibSp,Parch,Fare,Embarked_1,Embarked_2,Embarked_3,Embarked_4
0,1,0.0,1,0,0,0,-0.611972,1,0,-0.503176,1,0,0,0
1,2,1.0,0,1,0,1,0.630431,1,0,0.734809,0,1,0,0
2,3,1.0,1,0,0,1,-0.301371,0,0,-0.490126,1,0,0,0
3,4,1.0,0,1,0,1,0.397481,1,0,0.383263,1,0,0,0
4,5,0.0,1,0,0,0,0.397481,0,0,-0.487709,1,0,0,0


In [16]:
df_train = df[:len(df_train)]
df_test = df[len(df_train):]

X_train = df_train.drop(["PassengerId", "Survived"], axis=1)
y_train = df_train["Survived"]
X_test = df_test.drop(["PassengerId", "Survived"], axis=1)

In [17]:
X_train

,Pclass_1,Pclass_2,Pclass_3,Sex,Age,SibSp,Parch,Fare,Embarked_1,Embarked_2,Embarked_3,Embarked_4
0,1,0,0,0,-0.611972,1,0,-0.503176,1,0,0,0
1,0,1,0,1,0.630431,1,0,0.734809,0,1,0,0
2,1,0,0,1,-0.301371,0,0,-0.490126,1,0,0,0
3,0,1,0,1,0.397481,1,0,0.383263,1,0,0,0
4,1,0,0,0,0.397481,0,0,-0.487709,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,1,0,-0.223721,0,0,-0.392009,1,0,0,0
887,0,1,0,1,-0.844922,0,0,-0.063340,1,0,0,0
888,1,0,0,1,0.000000,1,2,-0.189974,1,0,0,0
889,0,1,0,0,-0.301371,0,0,-0.063340,0,1,0,0
